In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import a6
import typing as t
import xarray as xr
import math
import matplotlib.pyplot as plt
import numpy as np
import pathlib

In [ ]:
path = pathlib.Path("/home/fabian/Documents/MAELSTROM/gwl")

gwl = xr.open_dataset("../data/gwl.nc")
modes = a6.modes.methods.determine_lifetimes_of_modes(gwl["GWL"])

data = xr.open_dataset(
    "/home/fabian/Documents/MAELSTROM/bootcamp/data/ml_level_137_temperature_daily_mean_2017_2020.nc"
)
data_2 = xr.open_dataset(
    "/home/fabian/Documents/MAELSTROM/data/pca/pressure_level_500_950_daily_mean_2017_2020.nc"
).sel(level=500)
data_2["z_h"] = a6.features.methods.calculate_geopotential_height(data_2["z"])

In [ ]:
fig, _ = a6.plotting.plot_modes_durations(modes)
fig.savefig(path / "gwls.pdf")

In [ ]:
for mode in modes:
    datetimes = list(
        mode.get_dates(start=data["time"][0], end=data["time"][-1])
    )
    fig, _ = a6.plotting.plot_combined(
        data=data_2,
        dates=datetimes,
    )
    fig.savefig(path / f"gwl_{mode.label}.png")